In [85]:
import pandas as pd
import numpy as np
from math import sqrt

In [86]:
df = pd.read_csv('../src/results_fcfs-400-0,6.csv', delimiter=',', header=None, names=['batch', 'E_W', 'E_Nq', 'E_Ns'], skiprows=1)

In [87]:
df['E_W'] = df['E_W'].map(lambda x: float(x))
df['E_Nq'] = df['E_Nq'].map(lambda x: float(x))
df['E_Ns'] = df['E_Ns'].map(lambda x: float(x))


In [88]:
df

,batch,E_W,E_Nq,E_Ns
0,1,1.297820,0.7775,0.5625
1,2,1.408964,0.7925,0.6000
2,3,1.096503,0.7000,0.6325
3,4,1.754517,1.0700,0.6025
4,5,1.684256,0.9850,0.6675
...,...,...,...,...
3195,3196,1.209993,0.6000,0.5150
3196,3197,1.235442,0.6975,0.5900
3197,3198,1.166647,0.7775,0.5350
3198,3199,1.129580,0.6200,0.5400


In [89]:
avgs = []
avgs.append(df['E_W'].mean())
avgs.append(df['E_Nq'].mean())
avgs

[1.4950514041524676, 0.8989125000000001]

In [90]:
soma = np.zeros(2)
for index, row in df.iterrows():
    soma[0] += (row['E_W'] - avgs[0])**2
    soma[1] += (row['E_Nq'] - avgs[1])**2
var = soma * 1/(len(df) - 1)
display(var)

array([0.21989403, 0.12184603])

In [91]:
mean_width_ic = [sqrt(var[0])/sqrt(len(df)), sqrt(var[1])/sqrt(len(df))]
mean_IC_w = (avgs[0] - 1.96*mean_width_ic[0], avgs[0] + 1.96*mean_width_ic[0])
mean_IC_nq = (avgs[1] - 1.96*mean_width_ic[1], avgs[1] + 1.96*mean_width_ic[1])

In [92]:
print(f"IC da média de W: {mean_IC_w}")
print(f"IC da média de Nq: {mean_IC_nq}")

IC da média de W: (1.4788038572018631, 1.511298951103072)
IC da média de Nq: (0.8868180316428131, 0.9110069683571871)


In [93]:
mean_accuracy_w = (mean_IC_w[1] - mean_IC_w[0])/(mean_IC_w[1] + mean_IC_w[0])
mean_accuracy_nq = (mean_IC_nq[1] - mean_IC_nq[0])/(mean_IC_nq[1] + mean_IC_nq[0])

print(f"W mean precision: {mean_accuracy_w}\nNq mean precision: {mean_accuracy_nq}")

W mean precision: 0.010867550711284799
Nq mean precision: 0.01345455576286565


In [94]:
X2right = 19.023
X2left = 2.7
var_IC_w = (((len(df)-1) * var[0])/X2right, ((len(df)-1) * var[0])/X2left)
var_IC_nq = (((len(df)-1) * var[1])/X2right, ((len(df)-1) * var[1])/X2left)

In [95]:
print(f"IC da variância de W: {var_IC_w}")
print(f"IC da variância de Nq: {var_IC_nq}")

IC da variância de W: (36.97844709199788, 260.5337033448428)
IC da variância de Nq: (20.490220548809255, 144.36498722222163)


In [96]:
var_accuracy_w = (var_IC_w[1] - var_IC_w[0])/(var_IC_w[1] + var_IC_w[0])
var_accuracy_nq = (var_IC_nq[1] - var_IC_nq[0])/(var_IC_nq[1] + var_IC_nq[0])

print(f"W var precision: {var_accuracy_w}\nNq var precision: {var_accuracy_nq}")

W var precision: 0.7514155503383512
Nq var precision: 0.7514155503383511
